In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import time

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

### Scraping each result page

In [3]:
# URL of page to be scraped
url = 'https://www.indeed.com/jobs?q=data%20analyst&l=austin'
browser.visit(url)

In [4]:
title_list=[]
company_list=[]
location_list=[]
link_description_list=[]


for x in range(1, 75):
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.find_all('div', class_="jobsearch-SerpJobCard")
    
    
    for result in results:
    
        title=result.a['title']
        title_list.append(title)
        company=result.find('span', class_='company').text
        company_list.append(company)
        location=result.find(class_='location').text
        location_list.append(location)
        link_description=result.a['href']
        link_description_list.append(link_description)
        
        #print('page:', x, '-------------')
        #print(title)
        #print(company)
        #print(location)
        #print(link_description)
        #print("-------------------------")
        

    
    

### Scraping the description

In [7]:
description_url_list = ['http://indeed.com' + url for url in link_description_list]

In [8]:
description_text_list=[]
URL=[]
for urls in description_url_list:
    URL.append(urls)
    response = requests.get(urls)
    soup = BeautifulSoup(response.text, 'html.parser')
    description_results=soup.find_all('div', class_="jobsearch-JobComponent-description")
    for description in description_results:
        actual_description=description.text
    description_text_list.append(actual_description)
#         print(urls)
#         print(actual_description)
#         print('------------------------')
        
       

In [9]:
len(description_text_list)

1184

In [10]:
len(URL)

1184

In [11]:
len(description_url_list)

1184